## Load Libraries

In [1]:
import json
from random import randint, sample
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler

import xgboost
import treelite


sns.set(style="ticks")
np.set_printoptions(suppress=True)

pd.set_option('display.max_rows', 150)

## Load Data

In [36]:
with open('/data/data/NonwearCheck/450/Results/annotations.json', 'r') as f:
    annotations = json.load(f)

record_annotation_index = {}
for record_annotation in annotations['record_annotations']:
    if record_annotation['id'] not in record_annotation_index:
        record_annotation_index[record_annotation['id']] = {}
    id = record_annotation['id']
    record_annotation.pop('id')
    record_annotation_index[id].update(record_annotation)

segment_annotation_index = {}
for segment_annotation in annotations['segment_annotations']:
    if segment_annotation['id'] not in segment_annotation_index:
        segment_annotation_index[segment_annotation['id']] = {}
    id = segment_annotation['id']
    segment_annotation.pop('id')
    segment_annotation_index[id].update(segment_annotation)

FileNotFoundError: [Errno 2] No such file or directory: '/data/data/NonwearCheck/450/Results/annotations.json'

In [2]:
df_feats = pd.read_csv("/data/Results/df_feat_ppg_ir.csv", index_col=None)
df_feats = df_feats.iloc[shuffle(range(len(df_feats)), random_state=0), :]

df_objects = pd.read_csv("/data/Results/df_object_ppg_ir.csv", index_col=None)

In [3]:
feats_cols = [c for c in df_feats.columns if "ppg" in c]
target_col = "wear_category_id"

## 全数据构建模型

In [4]:
# 利用xgboost选择前50个特征分析
params = {'max_depth': 6, 'objective': 'binary:logistic', "n_jobs": -1}
num_iter = 20

X_cols, y_col = [
    'ppg-ir__number_peaks__n_3',
    'ppg-ir__number_peaks__n_10',
    'ppg-ir__autocorrelation__lag_2',
    'ppg-ir__agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"max"',
    'ppg-ir__agg_linear_trend__attr_"stderr"__chunk_len_10__f_agg_"mean"',
    'ppg-ir__agg_linear_trend__attr_"stderr"__chunk_len_10__f_agg_"max"',
    'ppg-ir__binned_entropy__max_bins_10',
    'ppg-ir__change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.8',
    'ppg-ir__change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.8'
                       ], target_col

X, y = df_feats.loc[:, X_cols].values, df_feats.loc[:, y_col].values

# scaler = MinMaxScaler()
# X = scaler.fit_transform(X)

D_train = xgboost.DMatrix(X, label=y)
D_test = xgboost.DMatrix(X, label=y)

bst = xgboost.train(params, D_train, num_iter, [(D_train, 'train')], verbose_eval=False)

accuracy_score(y, bst.predict(D_test) > 0.6)

[11:18:54] WARNING: /xgboost/src/learner.cc:1043: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.9800347609073489

In [5]:
model = treelite.Model.from_xgboost(bst)

model.export_srcpkg(platform='unix', toolchain='gcc', pkgpath='./model_20210816_002.zip',
                    libname='mymodel.so', verbose=True, params={'quantize': 1})

[11:19:03] /workspace/src/compiler/ast_native.cc:44: Using ASTNativeCompiler
[11:19:03] /workspace/src/compiler/ast/split.cc:24: Parallel compilation disabled; all member trees will be dumped to a single source file. This may increase compilation time and memory usage.
[11:19:03] /workspace/src/c_api/c_api.cc:286: Code generation finished. Writing code to files...
[11:19:03] /workspace/src/c_api/c_api.cc:291: Writing file recipe.json...
[11:19:03] /workspace/src/c_api/c_api.cc:291: Writing file header.h...
[11:19:03] /workspace/src/c_api/c_api.cc:291: Writing file main.c...


## 验证和C一致性

In [6]:
selected_object_ids = [5000]# list(range(3000, 3004))

### 待验证数据

In [10]:
selected_object_ids = [10999]
mask = np.isin(df_objects['id'], selected_object_ids)
mask = np.where(mask)[0]
df_objects.loc[mask, 'ppg-ir'].values

array([13230203, 13230173, 13229802, 13230442, 13228897, 13231315,
       13227935, 13230280, 13228136, 13231965, 13228368, 13229783,
       13228231, 13229994, 13229018, 13229701, 13229501, 13230017,
       13229313, 13229432, 13229118, 13230280, 13229857, 13229976,
       13228734, 13229642, 13229548, 13229988, 13229345, 13229248,
       13230175, 13228941, 13229246, 13229156, 13229100, 13229680,
       13228774, 13229980, 13229188, 13230208, 13228493, 13229794,
       13228541, 13229949, 13228552, 13229909, 13228546, 13229518,
       13229137, 13229255, 13229191, 13228707, 13229130, 13229288,
       13229336, 13228749, 13229527, 13229401, 13229928, 13229863,
       13228635, 13229641, 13229312, 13230201, 13228903, 13229414,
       13229466, 13229270, 13229348, 13228732, 13229308, 13229332,
       13229293, 13229271, 13229068, 13228229, 13229915, 13229749,
       13230379, 13228453, 13229118, 13228419, 13231276, 13228809,
       13229652, 13228644, 13229635, 13229153, 13229878, 13228

### 特征一致性

In [12]:
feats = df_feats.loc[selected_object_ids, X_cols]
feats

,ppg-ir__number_peaks__n_3,ppg-ir__number_peaks__n_10,ppg-ir__autocorrelation__lag_2,"ppg-ir__agg_linear_trend__attr_""intercept""__chunk_len_50__f_agg_""max""","ppg-ir__agg_linear_trend__attr_""stderr""__chunk_len_10__f_agg_""mean""","ppg-ir__agg_linear_trend__attr_""stderr""__chunk_len_10__f_agg_""max""",ppg-ir__binned_entropy__max_bins_10,"ppg-ir__change_quantiles__f_agg_""var""__isabs_False__qh_1.0__ql_0.8","ppg-ir__change_quantiles__f_agg_""var""__isabs_True__qh_1.0__ql_0.8"
10999,18.0,6.0,0.08966,8231.677333,0.026348,0.057512,1.642766,0.753176,0.260478


### 模型一致性

In [13]:
D = xgboost.DMatrix(feats.values)
bst.predict(D)

array([0.9985404], dtype=float32)

In [66]:
import json
import logging

_logger = logging.getLogger('nni')

In [67]:
_logger.info(
            "Creating graph json, writing to. Visualization enabled.")